In [357]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(font_scale=2)

In [665]:
# Parameters

R0 = 3.3
AR = 0.8
C0 = 100
POP = 1e7
gamma = 1

In [671]:
# SIR model https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model

def ds(s,i,pop=POP,r0=R0,gamma=gamma):
    return -s*i*r0*gamma/pop

def di(s,i,pop=POP,r0=R0,gamma=gamma):
    return -ds(s,i,pop,r0,gamma) - gamma*i

def dr(i,gamma=gamma):
    return gamma*i

def simulate(tmax,i0,s0,pop=POP,r0=R0,gamma=gamma,step_size=5e-5):
    ins = [i0]
    ss = [s0]
    for t in range(int(tmax/step_size)):
        ins.append(ins[-1] + step_size*di(ss[-1],ins[-1],pop,r0,gamma))
        ss.append(ss[-1] + step_size*ds(ss[-1],ins[-2],pop,r0,gamma))
    return np.vstack([ins, ss])


def hospital_stress_delta(delta,i0,pop=POP,r0=R0,thresh=2.5e-3,step_size=1e-5):
    re0 = i0/r0
    s1 = simulate(50,i0,pop-i0-re0,pop,r0,step_size=step_size)
    s2 = simulate(50,i0+delta,pop-i0-delta-re0,pop,r0,step_size=step_size)
    inds1, inds2 = s1[0]>=pop*thresh,s2[0]>=pop*thresh
    
    return  (s2[0][inds2]-pop*thresh).sum()*step_size - (s1[0][inds1]-pop*thresh).sum()*step_size

def infections_delta(delta,i0,pop=POP,r0=R0,thresh=2.5e-3,step_size=1e-5):
    re0 = i0/r0
    s1 = simulate(50,i0,pop-i0-re0,pop,r0,step_size=step_size)
    s2 = simulate(50,i0+delta,pop-i0-delta-re0,pop,r0,step_size=step_size)
    
    return  min(s1[1]) - min(s2[1])

In [673]:
rates = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1]

for rate in rates:
    print(f'Hospital stress 10 infections against {rate} background rate: {hospital_stress_delta(10,POP*rate)}\n\
            Additional infections same parameters: {infections_delta(10,POP*rate)}')

Hospital stress 10 infections against 1e-06 background rate: 4.900300201028585
            Additional infections same parameters: 0.49332934635458514
Hospital stress 10 infections against 1e-05 background rate: 4.90035025216639
            Additional infections same parameters: 0.493333057325799
Hospital stress 10 infections against 0.0001 background rate: 4.900884047150612
            Additional infections same parameters: 0.4933729491312988
Hospital stress 10 infections against 0.001 background rate: 4.906206000596285
            Additional infections same parameters: 0.49376952124293894
Hospital stress 10 infections against 0.01 background rate: 1.6483936794102192
            Additional infections same parameters: 0.49774160591186956
Hospital stress 10 infections against 0.1 background rate: 0.6969954576343298
            Additional infections same parameters: 0.5378707085619681
